In [1]:
!pip install -q ultralytics numpy==1.26.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curr

In [2]:
import os
os.environ['YOLO_NO_CHECKS'] = '1'  # skip AMP and other auto-checks

In [3]:
from ultralytics import YOLO
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import json

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#### **Dataset Configuration:**

In [4]:
def create_dataset_yaml(dataset_path, output_path='custom_dataset.yaml'):
    dataset_path = str(dataset_path).rstrip("/")
    
    yaml_content = f"""\
path: {dataset_path}
train: {dataset_path}/images/train
val: {dataset_path}/images/val
nc: 3
names:
  0: eyes_closed
  1: eyes_open
  2: yawning
"""
    with open(output_path, 'w') as f:
        f.write(yaml_content)

    return output_path


#### **Training:**

In [5]:
def train_model(
    dataset_path,
    model_name='yolov5nu.pt',
    epochs=100,
    batch_size=16,
    img_size=640,
    project='runs/drowsiness',
    name='exp'
):
    data_yaml = create_dataset_yaml(dataset_path)
    model = YOLO(model_name)

    training_config = {
        'data': data_yaml,
        'epochs': epochs,
        'imgsz': img_size,
        'batch': batch_size,
        'device': 0,  
        'project': project,
        'name': name,
        'exist_ok': True,
        'optimizer': 'AdamW',
        'lr0': 0.001,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3.0,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'dropout': 0.1,
        'patience': 20,
        'hsv_h': 0.015,
        'hsv_s': 0.4,
        'hsv_v': 0.4,
        'degrees': 10.0,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.1,
        'copy_paste': 0.0,
        'val': True,
        'plots': True,
        'save': True,
        'save_period': 10
    }

    results = model.train(**training_config)
    
    save_dir = Path(project) / name
    best_model = save_dir / 'weights' / 'best.pt'
    return model, results, best_model, save_dir

#### **Evaluate Model:**

In [6]:
def evaluate_model(model_path, data_yaml):
    model = YOLO(model_path)
    metrics = model.val(data=data_yaml, verbose=True)
    return metrics

#### **Plot Results:**

In [7]:
def plot_results(results_dir):
    results_csv = Path(results_dir) / 'results.csv'
    if not results_csv.exists():
        return
    
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()

    # Full training metrics plot
    fig, axes = plt.subplots(3, 2, figsize=(14, 12))
    plots = [
        ('train/box_loss', 'Box Loss (Train)', 'loss', 'lower'),
        ('train/cls_loss', 'Classification Loss (Train)', 'loss', 'lower'),
        ('metrics/mAP50(B)', 'mAP@0.5 (Validation)', 'mAP', 'upper'),
        ('metrics/mAP50-95(B)', 'mAP@0.5-0.95 (Validation)', 'mAP', 'upper'),
        ('metrics/precision(B)', 'Precision (Validation)', 'score', 'upper'),
        ('metrics/recall(B)', 'Recall (Validation)', 'score', 'upper'),
    ]
    for ax, (col, title, ylabel, best) in zip(axes.flat, plots):
        if col in df.columns:
            ax.plot(df['epoch'], df[col], linewidth=2)
            ax.set_xlabel('Epoch')
            ax.set_ylabel(ylabel)
            ax.set_title(title)
            ax.grid(True, alpha=0.3, linestyle='--')
            best_val = df[col].min() if best=='lower' else df[col].max()
            best_epoch = df[col].idxmin() if best=='lower' else df[col].idxmax()
            ax.scatter([best_epoch], [best_val], color='red', s=100)

    plt.tight_layout()
    plt.savefig(Path(results_dir)/'training_curves.png', dpi=300)
    plt.close()

#### **Generate JSON Report:**

In [8]:
def generate_report(model_path, results_dir, data_yaml):
    metrics = evaluate_model(model_path, data_yaml)
    plot_results(results_dir)

    report = {
        "model_path": str(model_path),
        "training_completed": True,
        "metrics": {
            "mAP_50": float(metrics.box.map50),
            "mAP_50_95": float(metrics.box.map),
            "precision": float(metrics.box.mp),
            "recall": float(metrics.box.mr)
        },
        "per_class_metrics": {
            "eyes_closed": {"mAP_50": float(metrics.box.maps[0]) if len(metrics.box.maps) > 0 else 0},
            "eyes_open": {"mAP_50": float(metrics.box.maps[1]) if len(metrics.box.maps) > 1 else 0},
            "yawning": {"mAP_50": float(metrics.box.maps[2]) if len(metrics.box.maps) > 2 else 0}
        }
    }

    with open(Path(results_dir)/'training_report.json', 'w') as f:
        json.dump(report, f, indent=2)
    
    return report

#### **Configuration:**

In [9]:
DATASET_PATH = '/kaggle/input/drowsiness-detection-augmented-data/augmented_dataset'
MODEL_NAME = 'yolov5nu.pt'  
EPOCHS = 100
BATCH_SIZE = 16
IMG_SIZE = 640
PROJECT = 'runs/drowsiness'
NAME = 'yolov5_final'

#### **Run Training:**

In [10]:
model, results, best_model, save_dir = train_model(
    dataset_path=DATASET_PATH,
    model_name=MODEL_NAME,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    project=PROJECT,
    name=NAME
)

Ultralytics 8.3.232 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom_dataset.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov5nu.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov5_final, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         38         38       0.97      0.954      0.992      0.772
           eyes_closed         13         13      0.915          1       0.99      0.732
             eyes_open         14         14          1      0.863       0.99        0.8
               yawning         11         11      0.995          1      0.995      0.784
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/runs/drowsiness/yolov5_final


#### **Save Report:**

In [11]:
yaml_path = "custom_dataset.yaml"
report = generate_report(best_model, save_dir, data_yaml=yaml_path)


Ultralytics 8.3.232 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5n summary (fused): 84 layers, 2,503,529 parameters, 0 gradients, 7.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 353.7±71.0 MB/s, size: 179.3 KB)
val: Scanning /kaggle/input/drowsiness-detection-augmented-data/augmented_dataset/labels/val... 38 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 38/38 654.5it/s 0.1s
WARNING ⚠️ val: Cache directory /kaggle/input/drowsiness-detection-augmented-data/augmented_dataset/labels is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.7it/s 1.1s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         38         38       0.97      0.954      0.992      0.772
           eyes_closed         13         13      0.915          1       0.99      0.732
             eyes_open         14         14          1      0.863       0.99      0.801
               yawning         11         11      0.995          1      0.995      0.784
Speed: 4.4ms preprocess, 10.0ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /kaggle/working/runs/detect/val
